In [2]:
import json
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, roc_auc_score, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

### Added Features:
- `readout_match`: Binary feature; 1 if the logical readout (L) matches the expected readout, otherwise 0.
- `index_flip`: Eight columns (index_flip_0 to index_flip_7), representing the number of bit flips for each index between the three rounds.
- `agreement`: Proportion of bits that agree across all three rounds for each row.
- `flip_errs`: Count of the number of errors (1s) in the last four bits (Z3, Z2, Z1, Z0) of each round, where these bits should always be zero.

In [3]:
# File paths for the JSON files
json_files = [
    'data/json/qubit1Z_d3_a1_results.json',
    'data/json/qubit1Z_d3_a2_results.json',
    'data/json/qubit1Z_d3_b1_results.json'
]

# Output file path for the concatenated results
output_file = 'data/json/qubit1Z_d3_combined_results.json'

# Function to load a JSON file
def load_json_data(file_path):
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return {}

# Main script to concatenate JSON files
def combine_data(json_files, output_file) : 
    # Create an empty dictionary to store the concatenated data
    combined_data = {}

    # Loop over each JSON file and update the combined data
    for file_path in json_files:
        json_data = load_json_data(file_path)
        
        # Merge the json_data with combined_data
        for backend, data in json_data.items():
            if backend in combined_data:
                combined_data[backend].update(data)  # Update existing backend data
            else:
                combined_data[backend] = data  # Add new backend data
    
    # Save the combined data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(combined_data, f, indent=4)

    print(f"Data has been successfully concatenated into {output_file}.")

In [4]:
def json_to_expanded_df(json_file, d = 5, num_rounds = 3):
    with open(json_file, 'r') as f:
        data = json.load(f)

    def process_data(data):
        rows = []
        for backend, results in data.items():
            for key, count in results.items():
                split_data = key.split()  # Split based on space
                readout = int(split_data[0])  # L value (Z readout)
                bitstrings = split_data[1:]  # Remaining are the rounds

                # Flatten the bitstrings into individual bits
                expanded_bits = [int(bit) for bitstring in bitstrings for bit in bitstring]
                
                # Append backend, expanded bits, readout, and count
                rows.append([backend] + expanded_bits + [readout, count])

        return rows

    # Dynamically generate column names based on the bitstring length
    measure_bits = (d*d)-1  # 24 bits per round of d=5

    columns = ['backend']
    for r in range(num_rounds):
        for q in range(measure_bits):
            columns.append(f'bit_{q}_round_{r}')

    columns += ['z_readout', 'count']
    return pd.DataFrame(process_data(data), columns=columns)

def calculate_index_flip(row, measure_bits):
    """Calculate the number of flips for each bit between rounds."""
    flips = []
    
    # Iterate through the measure_bits bits 
    for i in range(measure_bits):
        # Check flips between round_0, round_1, and round_2
        bit_r0 = row[f'bit_{i}_round_0']
        bit_r1 = row[f'bit_{i}_round_1']
        bit_r2 = row[f'bit_{i}_round_2']
        
        # Count the flips between consecutive rounds
        flip_count = (bit_r0 != bit_r1) + (bit_r1 != bit_r2)
        flips.append(flip_count)
    
    return flips

def calculate_agreement(row, measure_bits):
    """Calculate the agreement between rounds for each bit."""
    agreement_count = 0

    # Iterate through the measure_bits bits and check if they agree across rounds
    for i in range(measure_bits):
        bit_r0 = row[f'bit_{i}_round_0']
        bit_r1 = row[f'bit_{i}_round_1']
        bit_r2 = row[f'bit_{i}_round_2']
        
        # Check if the bits are the same in all rounds
        if bit_r0 == bit_r1 == bit_r2:
            agreement_count += 1

    return agreement_count / measure_bits  # Proportion of agreement

def calculate_flip_errs(row, measure_bits):
    """
    Calculate the number of errors (1s) in the last measure_bits/2 bits of each round.

    Parameters:
    row: The row containing the bits data.
    measure_bits: The total number of measurement bits. 
                  The function will check the last measure_bits/2 bits for errors.
    """
    flip_errs = 0
    half_measure_bits = measure_bits // 2  # Calculate half of measure_bits

    # Iterate through the last half_measure_bits bits for each round
    for i in range(measure_bits - half_measure_bits, measure_bits):
        flip_errs += row[f'bit_{i}_round_0']
        flip_errs += row[f'bit_{i}_round_1']
        flip_errs += row[f'bit_{i}_round_2']

    return flip_errs

In [5]:
def build_backend_classifier(df, expected_readout=0, d=5):
    measure_bits = (d*d)-1
    
    df = df[df['backend'] != 'ibm_sherbrooke_a2'].copy()  # Ensure we're working on a copy
    df = df[df['backend'] != 'ibm_brisbane_a2'].copy()
    df = df[df['backend'] != 'ibm_kyiv_a2'].copy()
    
    # readout_match (1 if z_readout matches expected_readout, else 0)
    df['readout_match'] = (df['z_readout'] == expected_readout).astype(int)
    
    # index_flip (how many times each bit flips between rounds)
    index_flip_cols = [f'index_flip_{i}' for i in range(measure_bits)]
    
    # index_flip_cols (flips between rounds)
    df[index_flip_cols] = df.apply(lambda row: calculate_index_flip(row, measure_bits), axis=1, result_type='expand')

    # agreement (proportion of bits that agree across rounds)
    df['agreement'] = df.apply(lambda row: calculate_agreement(row, measure_bits), axis=1)

    # flip_errs (count of errors in the last measure_bits/2 bits of each round)
    df['flip_errs'] = df.apply(lambda row: calculate_flip_errs(row, measure_bits), axis=1)
    
    # Select features for training (excluding backend and count)
    X = df.drop(columns=['backend', 'count'])
    y = df['backend']  # Target
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Create and train the RandomForestClassifier
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    
    # Predictions and evaluation
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Output accuracy and classification report
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    return clf, df, y_test, y_pred

In [6]:
def build_svm_classifier(df, target_backend, expected_readout=0, d=5, adjust_threshold=False, threshold=0.5):
    measure_bits = (d*d)-1

    df = df[df['backend'] != 'ibm_sherbrooke_a2'].copy()  # Ensure we're working on a copy
    df = df[df['backend'] != 'ibm_brisbane_a2'].copy()
    df = df[df['backend'] != 'ibm_kyiv_a2'].copy()

    # readout_match (1 if z_readout matches expected_readout, else 0)
    df['readout_match'] = (df['z_readout'] == expected_readout).astype(int)

    # index_flip (how many times each bit flips between rounds)
    index_flip_cols = [f'index_flip_{i}' for i in range(measure_bits)]
    df[index_flip_cols] = df.apply(lambda row: calculate_index_flip(row, measure_bits), axis=1, result_type='expand')

    # agreement (proportion of bits that agree across rounds)
    df['agreement'] = df.apply(lambda row: calculate_agreement(row, measure_bits), axis=1)

    # flip_errs (count of errors in the last measure_bits/2 bits of each round)
    df['flip_errs'] = df.apply(lambda row: calculate_flip_errs(row, measure_bits), axis=1)

    # Convert the 'backend' column to binary (1 for target_backend, 0 for others)
    df['binary_backend'] = (df['backend'] == target_backend).astype(int)

    # Check class distribution before resampling
    print("Class distribution before resampling:")
    print(df['binary_backend'].value_counts())

    # Select features for training (excluding original backend and count columns)
    X = df.drop(columns=['backend', 'count', 'binary_backend'])
    y = df['binary_backend']  # Binary target variable

    # Apply SMOTE to oversample the minority class (1)
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Check class distribution after resampling
    print("Class distribution after resampling:")
    print(y_resampled.value_counts())

    # Split data using stratified sampling to maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled, random_state=42)

    # Ensure there are two classes present in training data
    print("Training set class distribution:", y_train.value_counts())
    print("Test set class distribution:", y_test.value_counts())

    if len(y_train.unique()) == 1:
        raise ValueError("Training set only contains one class. Check the distribution or increase dataset size.")

    # Create and train the SVM with class_weight='balanced'
    svm = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
    print('Created SVM...')
    svm.fit(X_train, y_train)
    print('\tFit SVM')

    # Get probability predictions
    y_proba = svm.predict_proba(X_test)[:, 1]  # Probability for the positive class

    # Apply custom threshold if needed
    if adjust_threshold:
        y_pred = (y_proba >= threshold).astype(int)
    else:
        y_pred = svm.predict(X_test)

    print('Got y pred')
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Print evaluation metrics
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Plot Precision-Recall Curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    plt.plot(recall, precision, marker='.', label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

    return svm, df, y_test, y_pred

In [7]:
def build_svm_classifier(df, target_backend, expected_readout=0, d=5, adjust_threshold=False, threshold=0.5):
    measure_bits = (d*d)-1

    df = df[df['backend'] != 'ibm_sherbrooke_a2'].copy()  # Ensure we're working on a copy
    df = df[df['backend'] != 'ibm_brisbane_a2'].copy()
    df = df[df['backend'] != 'ibm_kyiv_a2'].copy()

    # readout_match (1 if z_readout matches expected_readout, else 0)
    df['readout_match'] = (df['z_readout'] == expected_readout).astype(int)

    # index_flip (how many times each bit flips between rounds)
    index_flip_cols = [f'index_flip_{i}' for i in range(measure_bits)]
    df[index_flip_cols] = df.apply(lambda row: calculate_index_flip(row, measure_bits), axis=1, result_type='expand')

    # agreement (proportion of bits that agree across rounds)
    df['agreement'] = df.apply(lambda row: calculate_agreement(row, measure_bits), axis=1)

    # flip_errs (count of errors in the last measure_bits/2 bits of each round)
    df['flip_errs'] = df.apply(lambda row: calculate_flip_errs(row, measure_bits), axis=1)

    # Convert the 'backend' column to binary (1 for target_backend, 0 for others)
    df['binary_backend'] = (df['backend'] == target_backend).astype(int)

    # Check class distribution before splitting
    print("Class distribution before splitting:")
    print(df['binary_backend'].value_counts())

    # Select features for training (excluding original backend and count columns)
    X = df.drop(columns=['backend', 'count', 'binary_backend'])
    y = df['binary_backend']  # Binary target variable

    # Split data using stratified sampling to maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Check class distribution in the training and testing sets
    print("Training set class distribution before resampling:", y_train.value_counts())
    print("Test set class distribution:", y_test.value_counts())

    # Apply SMOTE to oversample the minority class only on the training data
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Check class distribution after resampling
    print("Training set class distribution after resampling:")
    print(y_train_resampled.value_counts())

    # Ensure there are two classes present in the resampled training data
    if len(y_train_resampled.unique()) == 1:
        raise ValueError("Training set only contains one class. Check the distribution or increase dataset size.")

    # Create and train the SVM with class_weight='balanced'
    svm = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
    print('Created SVM...')
    svm.fit(X_train_resampled, y_train_resampled)
    print('\tFit SVM')

    # Get probability predictions on the test set (real data, no resampling)
    y_proba = svm.predict_proba(X_test)[:, 1]  # Probability for the positive class

    # Apply custom threshold if needed
    if adjust_threshold:
        y_pred = (y_proba >= threshold).astype(int)
    else:
        y_pred = svm.predict(X_test)

    print('Got y pred')
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Print evaluation metrics
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Plot Precision-Recall Curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    plt.plot(recall, precision, marker='.', label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

    return svm, df, y_test, y_pred

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

def build_mlp_classifier(df, target_backend, expected_readout=0, d=5, adjust_threshold=False, threshold=0.5):
    measure_bits = (d * d) - 1

    df = df[df['backend'] != 'ibm_sherbrooke_a2'].copy()  # Ensure we're working on a copy
    df = df[df['backend'] != 'ibm_brisbane_a2'].copy()
    df = df[df['backend'] != 'ibm_kyiv_a2'].copy()

    # readout_match (1 if z_readout matches expected_readout, else 0)
    df['readout_match'] = (df['z_readout'] == expected_readout).astype(int)

    # index_flip (how many times each bit flips between rounds)
    index_flip_cols = [f'index_flip_{i}' for i in range(measure_bits)]
    df[index_flip_cols] = df.apply(lambda row: calculate_index_flip(row, measure_bits), axis=1, result_type='expand')

    # agreement (proportion of bits that agree across rounds)
    df['agreement'] = df.apply(lambda row: calculate_agreement(row, measure_bits), axis=1)

    # flip_errs (count of errors in the last measure_bits/2 bits of each round)
    df['flip_errs'] = df.apply(lambda row: calculate_flip_errs(row, measure_bits), axis=1)

    # Convert the 'backend' column to binary (1 for target_backend, 0 for others)
    df['binary_backend'] = (df['backend'] == target_backend).astype(int)

    # Check class distribution before splitting
    print("Class distribution before splitting:")
    print(df['binary_backend'].value_counts())

    # Select features for training (excluding original backend and count columns)
    X = df.drop(columns=['backend', 'count', 'binary_backend'])
    y = df['binary_backend']  # Binary target variable

    # Split data using stratified sampling to maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Check class distribution in the training and testing sets
    print("Training set class distribution before resampling:", y_train.value_counts())
    print("Test set class distribution:", y_test.value_counts())

    # Apply SMOTE to oversample the minority class only on the training data
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Check class distribution after resampling
    print("Training set class distribution after resampling:")
    print(y_train_resampled.value_counts())

    # Ensure there are two classes present in the resampled training data
    if len(y_train_resampled.unique()) == 1:
        raise ValueError("Training set only contains one class. Check the distribution or increase dataset size.")

    # Create and train the MLP Classifier
    mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
    print('Created MLP...')
    mlp.fit(X_train_resampled, y_train_resampled)
    print('\tFit MLP')

    # Get probability predictions on the test set (real data, no resampling)
    y_proba = mlp.predict_proba(X_test)[:, 1]  # Probability for the positive class

    # # Apply custom threshold if needed
    # if adjust_threshold:
    #     y_pred = (y_proba >= threshold).astype(int)
    # else:
    #     y_pred = mlp.predict(X_test)

    print('Got y pred')
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # Calculate FPR and FNR
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate

    # Print evaluation metrics
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}")
    print(f"False Positive Rate (FPR): {fpr:.2f}")
    print(f"False Negative Rate (FNR): {fnr:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Plot Precision-Recall Curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    plt.plot(recall, precision, marker='.', label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

    return mlp, df, y_test, y_pred

In [11]:
def get_backend_results(df, backend, suffix, d=5):
    target_backend = f'{backend}{suffix}'
    print(f"Classifying for backend: {target_backend}")
    clf, df, y_test, y_pred = build_mlp_classifier(df, target_backend=target_backend, expected_readout=0, d=d)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {target_backend}: {accuracy * 100:.2f}%")
    
    return clf, y_test, y_pred

def plot_confusion_matrices(results, filename):
    plt.figure(figsize=(15, 10))
    
    for i, result in enumerate(results):
        plt.subplot(3, 3, i + 1)
        cm = confusion_matrix(result['y_test'], result['y_pred'])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap='magma', ax=plt.gca(), colorbar=False)
        plt.title(result['backend'])
    
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    
def get_results_all_backends(backends = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv'], suffixes = ['_a1', '_b1']):
    results = []

    json_file = 'data/json/qubit1Z_d5_combined_results.json'
    df = json_to_expanded_df(json_file, d=5)

    # Loop through backends and suffixes
    for backend in backends:
        for suffix in suffixes:
            mlp, df, y_test, y_pred = get_backend_results(df, backend, suffix, d=5)
            results.append({
                'backend': f'{backend}{suffix}',
                'mlp': mlp,
                'y_test': y_test,
                'y_pred': y_pred
            })
            
    plot_confusion_matrices(results, 'qubit1Z_d5_binary_smote.png')

In [12]:
get_results_all_backends()

Classifying for backend: ibm_brisbane_a1
Class distribution before splitting:
binary_backend
0    30000
1     5000
Name: count, dtype: int64
Training set class distribution before resampling: binary_backend
0    21000
1     3500
Name: count, dtype: int64
Test set class distribution: binary_backend
0    9000
1    1500
Name: count, dtype: int64
Training set class distribution after resampling:
binary_backend
0    21000
1    21000
Name: count, dtype: int64
Created MLP...
	Fit MLP
Got y pred


NameError: name 'y_pred' is not defined

## d=3

In [ ]:
# qubit1Z_d3_state.pkl : qubit1Z_d3_{backend}_transpiled.qpy (at time of creation)
json_file = 'data/json/qubit1Z_d3_results.json'
df = json_to_expanded_df(json_file, d=3)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=3)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
# qubit1Z_d3_state.pkl : qubit1Z_d3_{backend}_transpiled.qpy (1 week later)
json_file = 'data/json/qubit1Z_d3_2_results.json'
df = json_to_expanded_df(json_file, d=3)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=3)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
# qubit1Z_d3_state.pkl : qubit1Z_d3_{backend}_transpiled.qpy (1 week later)
json_file = 'data/json/qubit1Z_d3_b_results.json'
df = json_to_expanded_df(json_file, d=3)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=3)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
json_file = 'data/json/qubit1Z_d3_combined_results.json'
df = json_to_expanded_df(json_file, d=3)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=3)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')
plt.xticks(rotation=90)
plt.savefig('qubit1Z_d3_combined_matrix_diffmaps.png', dpi=300, bbox_inches='tight')
plt.show()

## d=5

In [ ]:
# qubit1Z_d5_state.pkl : qubit1Z_d5_{backend}_transpiled.qpy (at time of creation)
json_file = 'data/json/qubit1Z_d5_results.json'
df = json_to_expanded_df(json_file, d=5)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=5)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
# qubit1Z_d5_{backend}_transpiled.qpy (approx 1 week after its creation)
json_file = 'data/json/qubit1Z_d5_2_results.json'
df = json_to_expanded_df(json_file, d=5)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=5)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
# qubit1Z_d5_{backend}_transpiled.qpy (approx 1 week after its creation)
json_file = 'data/json/qubit1Z_d5_b_results.json'
df = json_to_expanded_df(json_file, d=5)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=5)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')

In [ ]:
json_file = 'data/json/qubit1Z_d5_combined_results.json'
df = json_to_expanded_df(json_file, d=5)
clf, updated_df, y_test, y_pred = build_backend_classifier(df, d=5)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap='magma', normalize='true')
plt.xticks(rotation=90)
plt.savefig('qubit1Z_d5_combined_matrix_diffmaps.png', dpi=300, bbox_inches='tight')

In [ ]:
json_file = 'data/json/qubit1Z_d5_combined_results.json'
df = json_to_expanded_df(json_file, d=5)
#clf, df, y_test, y_pred = build_binary_backend_classifier(df, expected_readout=0, d=5, target_backend='ibm_brisbane_a1')

In [ ]:
# switch to mlp

- increase # of mappings to classify
- classify based on "older" transpilations / qpys
- change to binary classification
- mappings would only matter for qPuf, train one classifier per mapping

- rerun saved transpilations from d=5 tomorrow, next week
- implement arbitrary mapping?
- generate confusion matrix across machines and mappings
- gernerate a new classifier per mapping => you expect to know which mapping you're working with
    - pull the classifier that you generated for that backend & mapping
    - ex: have brisbane mapping a, everything is labeled as brisbane_a or not

```python
for backend in backends:
    for mapping_label in ["a", "b", "c"]:
        random_layout = np.random.permutation(backend.num_qubits)[
            : qc.num_qubits
        ]
        qc_transpiled = transpile(qc, backend, initial_layout=random_layout)
```